In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 트랜스포머(Transformer)란?

인공신경망 알고리즘은 크게, 합성곱 신경망(CNN), 순환 신경망(RNN), 트랜스포머(Transformer) 3가지로 나눠집니다.

이 중 트랜스포머는, 2017년 구글이 발표한 논문인 "Attention is all you need"에서 나온 모델로, 셀프 에텐션(Self-Attention)이라는 방식을 사용하는 모델입니다.

트랜스포머는, 이러한 어텐션 방식을 사용해, 문장 전체를 병렬구조로 번역할 뿐만 아니라, 멀리 있는 단어까지도 연관성을 만들어 유사성을 높였으며, RNN의 한계를 극복했습니다.

또한, 이미지나 언어 번역에 폭넓게 쓰이고 있으며, GPT-3, BERT 등이 가장 관심을 많이 받고 있는 모델입니다.

- Hugging Face 란?

'허깅 페이스’는, 자연어 처리 스타트업이 개발한,
다양한 트랜스포머 모델(transformer.models)과 학습 스크립트(transformer.Trainer)를 제공하는 모듈입니다.

허깅 페이스를 사용한다면, 트랜스포머 모델 사용시 layer, model 등을 선언하거나 학습 스크립트를 구현해야하는 수고를 덜 수 있습니다.

참고 링크
https://github.com/huggingface/transformers

일반적인 layer.py, model.py 는 transformer.models 로,
train.py 는 transformer.Trainer 로 대응해서 사용할 수 있습니다.

- transformers.models

트랜스포머 기반의 다양한 모델을 pytorch, tensorflow 로 각각 구현해놓은 모듈입니다.

각 모델에 맞는 tokenizer 도 구현되어 있습니다.

- transformers.Trainer

딥러닝 학습 및 평가에 필요한 optimizer, weight updt, learning rate schedul, ckpt, tensorbord, evaluation 등을 수행하는 모듈입니다.

Trainer.train 함수를 호출하면, 이 모든 과정이, 사용자가 원하는 arguments에 맞게 실행됩니다.

pytorch lightning 과 비슷하게, 공통적으로 사용되는 학습 스크립트를 모듈화 하여 편하게 사용할 수 있다는 점이 장점입니다.

- 결론

기존 pytorch 학습 스크립트에서 반복되는 부분(optimizer, lr schedul, tensorbord, gpu 병렬 처리,..)을 따로 구현하지 않고, arguments 로 편하게 통제할 수 있습니다.
다양한 트랜스포머 기반 모델 구현체들을, 손쉽게 당겨 쓸 수 있습니다.

high level로 모듈화 되어 있기 때문에, 커스터마이징이 비교적 어렵습니다.

커스터마이징은 소스코드를 참고하여 원하는 class를 상속 받아 overiding 하면 됩니다.

In [2]:
!pip install pytorch
!pip install folium==0.2.1
!pip install datasets
!pip install transformers # cnn, rnn
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-h6iqa5bb/pytorch_7e68456391a04d7d89e0dea0ce5fadd0/setup.py'"'"'; __file__='"'"'/tmp/pip-install-h6iqa5bb/pytorch_7e68456391a04d7d89e0dea0ce5fadd0/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-r4j6vm25/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pytorch Check the logs f

In [ ]:
pip install wandb # Weights & Biases(WandB)는 실시간으로 훈련을 모니터링할 수 있는 파이썬 패키지입니다.

In [6]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.8 MB/s 


In [11]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 텐서 모델
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa

# transformers 모델
import transformers
from transformers import AutoTokenizer
from transformers import TFAutoModel, TFAutoModelForSequenceClassification
transformers.logging.set_verbosity_error()

# 경고메세지
import warnings
warnings.filterwarnings('ignore')

# Weights & Biases(WandB)는 실시간으로 훈련을 모니터링
import argparse
import wandb
from wandb.keras import WandbCallback
wandb.init(project="DACON_235938", name="pretrained_emb")

 # 파서
parser = argparse.ArgumentParser(description="pretrained_emb")
# kosroberta or koelectra_base or roberta
parser.add_argument('--pretrained_model', default="roberta", type=str)
parser.add_argument('--feature_name', default="reviews", type=str) # reviews or tokenized_stem
parser.add_argument('--optimizer', default="sgd", type=str) # sgd or adam
parser.add_argument('--learning_rate', default=0.001, type=float)
parser.add_argument('--loss', default="cc", type=str)
parser.add_argument('--label_smoothing', default=0.1, type=float)
parser.add_argument('--batch_size', default=16, type=int)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--validation_size', default=0.1, type=float)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

# 사용할 모델 3가지
if args.pretrained_model == "kosroberta":
    pretrained_model = "jhgan/ko-sroberta-multitask"
if args.pretrained_model == "koelectra":
    pretrained_model = "monologg/koelectra-base-v3-discriminator"
if args.pretrained_model == "roberta":
    pretrained_model = "klue/roberta-large"
wandb.config.update(args)


tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

# 매개변수
feature_name = args.feature_name
optimizer = args.optimizer
learning_rate = args.learning_rate
loss = args.loss
label_smoothing = args.label_smoothing
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
validation_size = args.validation_size
seed = args.seed

# 시드 고정
def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()

train = pd.read_csv("./drive/MyDrive/input/쇼핑몰 리뷰 평점 분류/train.csv")
test = pd.read_csv("./drive/MyDrive/input/쇼핑몰 리뷰 평점 분류/test.csv")
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [12]:
train[feature_name]

0                                           조아요 처음구입 싸게햇어요
1        생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...
2        디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...
3        기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...
4                                        튼튼하고 손목을 잘 받쳐주네요~
                               ...                        
24995                  일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여
24996          동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.
24997                              향이 좋아요!! 다른 향도 시켜보고 싶어요
24998            펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다
24999                                     호호애미 제품은 늘 믿고 써요
Name: reviews, Length: 25000, dtype: object

In [ ]:
# feature_name = 'reviews'
# train[feature_name][0] = '조아요 처음구입 싸게햇어요'
# tokenizer(train[feature_name][0]) = {'input_ids': [0, 15015, 2182, 3790, 2251, 2372, 1336, 2318, 2749, 10283, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
# tokenizer(train[feature_name][0])["input_ids"] = [0, 15015, 2182, 3790, 2251, 2372, 1336, 2318, 2749, 10283, 2]

tokenizer.convert_ids_to_tokens(tokenizer(train[feature_name][0])["input_ids"])

['[CLS]', '조아', '##요', '처음', '##구', '##입', '싸', '##게', '##햇', '##어요', '[SEP]']

In [19]:
test[feature_name]

0                                           채소가 약간 시들어 있어요
1        발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2                                 부들부들 좋네요 입어보고 시원하면 또 살게요
3            이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4             검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요
                               ...                        
24995                                         사용해보니 좋아요~^^
24996                   저렴한가격에. 질좋고. 핏좋고. 너무. 이쁘게. 입고다녀요..
24997    세트상품이라고 써있어서 그런줄 알고 구매했더니 단품이었네요 낚인 느낌도 들고 그러네...
24998                                       역시 로네펠트!! 좋아요.
24999    데싱 디바 써보고 갠찮아서 비슷 한줄 알앗더니 완전 별로예요ㅡㅡ3000언 더주고 디...
Name: reviews, Length: 25000, dtype: object

In [31]:
X = train[feature_name] # 25000개의 리뷰
X_test = test[feature_name] # 25000개의 리뷰 
y = train["target"] # 별점
y.value_counts() # 별점개수

5    10000
2     8000
1     4500
4     2500
Name: target, dtype: int64

In [32]:
# 별점 재설정 [1,2,4,5] -> [0,1,2,3]
label_unique = sorted(np.unique(y)) # [1, 2, 4, 5]
label_unique = {key : value for key, value in zip(label_unique, range(len(label_unique)))}
y = np.array([label_unique[k] for k in y]) # array([1, 0, 1, ..., 2, 1, 3])
label_unique

{1: 0, 2: 1, 4: 2, 5: 3}

In [24]:
train[feature_name].apply(tokenizer.tokenize)

0               [조아, ##요, 처음, ##구, ##입, 싸, ##게, ##햇, ##어요]
1        [생각, ##보, ##다, 잘, 안, ##돼, ##요, 매, ##지, 바른, ##지...
2        [디자인, ##은, ##괜, ##찮, ##은, ##데, 상품, ##이, 금, ##이...
3        [기전, ##에, 이, 제품, ##말, ##고, 이마트, 트레이, ##더스, ##에...
4             [튼튼, ##하고, 손목, ##을, 잘, 받쳐, ##주, ##네, ##요, ~]
                               ...                        
24995    [일단, ##향, ##이너, ##무, ##너무, ##좋, ##은, ##데, ##지,...
24996    [동일, ##한, 업체, ##에서, 발송, ##하고, 한, 상자, ##로, 배송, ...
24997    [향, ##이, 좋아, ##요, !, !, 다른, 향, ##도, 시켜, ##보, #...
24998    [펼쳐, ##보, ##니, 끝, ##부분, ##쯤, ##에, 50, ##c, ##m...
24999                   [호호, ##애미, 제품, ##은, 늘, 믿, ##고, 써요]
Name: reviews, Length: 25000, dtype: object

In [26]:
train["len"] = train[feature_name].apply(tokenizer.tokenize).apply(len) # 각 리뷰별 토큰 길이
test["len"] = test[feature_name].apply(tokenizer.tokenize).apply(len) # 각 리뷰별 토큰 길이

# sns.histplot(train["len"])
# plt.show()
# sns.histplot(test["len"])
# plt.show()
# sns.histplot(train["len"][train["target"]==1])
# plt.show()
# sns.histplot(train["len"][train["target"]==2])
# plt.show()
# sns.histplot(train["len"][train["target"]==4])
# plt.show()
# sns.histplot(train["len"][train["target"]==5])
# plt.show()

MAX_LENGTH = train["len"].max() # 100개

In [39]:
class BertDataGenerator(tf.keras.utils.Sequence): # X_train.values, y_train, batch_size=BATCH_SIZE, shuffle=True
    def __init__(self, sentence, labels, batch_size=BATCH_SIZE, shuffle=True, include_targets=True):
        self.sentence = sentence # X_train.values, X_val.values
        self.labels = labels # y_train, y_val
        self.shuffle = shuffle # True, False
        self.batch_size = batch_size # 16, 16
        self.include_targets = include_targets
        self.tokenizer = tokenizer
        self.indexes = np.arange(len(self.sentence))
        self.on_epoch_end()

    def __len__(self):
        return len(self.sentence) // self.batch_size

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence = self.sentence[indexes]

        # 인코더
        encoded = self.tokenizer.batch_encode_plus(sentence.tolist(),
                                                   add_special_tokens=True,
                                                    padding="max_length",
                                                    truncation=True,
                                                    max_length=MAX_LENGTH,
                                                    return_tensors="tf",
                                                    return_token_type_ids=True,
                                                    return_attention_mask=True)

        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.RandomState(seed).shuffle(self.indexes)

In [40]:
# 버트용 3개 데이터
input_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name="input_ids") # 실제 토큰값([2, 19017, 8482, 3])/ <KerasTensor: shape=(None, 100) dtype=int32 (created by layer 'input_ids')>
attention_masks = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name="attention_masks") # 실제 토큰이 자리하는지 아닌지(0,1) / <KerasTensor: shape=(None, 100) dtype=int32 (created by layer 'attention_masks')> 
token_type_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name="token_type_ids") # 파인튜닝을 실시할 때는 모두 0 / <KerasTensor: shape=(None, 100) dtype=int32 (created by layer 'token_type_ids')>

# 버트 모델(사전학습)
from transformers import TFAutoModel, TFAutoModelForSequenceClassification
bert_model = TFAutoModel.from_pretrained(pretrained_model, from_pt=True)
bert_model.trainable = False
bert_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)

# 버트 적용 모델
x = bert_output.last_hidden_state # <KerasTensor: shape=(None, 100, 1024) dtype=float32 (created by layer 'tf_roberta_model')>
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
output = layers.Dense(4, activation="softmax")(x)
model = tf.keras.models.Model(inputs=[input_ids, attention_masks, token_type_ids], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 100)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  336656384  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_masks[0][0]',    

In [41]:
# Bert모델 데이터
y = tf.keras.utils.to_categorical(y) # 원핫인코딩
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_size, random_state=seed, stratify=y) # validation_size=0.1

train_ds = BertDataGenerator(X_train.values, y_train, batch_size=BATCH_SIZE, shuffle=True) # BATCH_SIZE=16 # 22500
val_ds = BertDataGenerator(X_val.values, y_val, batch_size=BATCH_SIZE, shuffle=False) # 2500

In [ ]:
# 매개변수
lr = tf.keras.optimizers.schedules.CosineDecay(learning_rate, decay_steps=1000)
if args.optimizer == "sgd":
    optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
elif args.optimizer == "adam":
    optim = tf.keras.optimizers.Adam(learning_rate=lr)
    
if loss == "cc":
    # loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
    loss_function = tf.keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing)
elif loss == "fl":
    loss_function = tfa.losses.SigmoidFocalCrossEntropy()
    
# 모델 실행
model.compile(optimizer=optim, loss=loss_function, metrics=["accuracy"])
# 파일경로
checkpoint_filepath=f"load_model/{parser.description}"
# 조기중단/체크 포인트
checkpoint_callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                       tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor="val_loss", save_best_only=True, save_weights_only=True)]

history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[checkpoint_callback, WandbCallback()]) # EPOCHS=100

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/100
1406/1406 [==============================] - ETA: 0s - loss: 1.1801 - accuracy: 0.5425

INFO:tensorflow:Assets written to: /content/wandb/run-20220805_072013-2zn9kmhx/files/model-best/assets


INFO:tensorflow:Assets written to: /content/wandb/run-20220805_072013-2zn9kmhx/files/model-best/assets
wandb: Adding directory to artifact (/content/wandb/run-20220805_072013-2zn9kmhx/files/model-best)... Done. 8.1s


1406/1406 [==============================] - 510s 346ms/step - loss: 1.1801 - accuracy: 0.5425 - val_loss: 1.1374 - val_accuracy: 0.5601 - _timestamp: 1659684473.3303 - _runtime: 454.6312
Epoch 2/100
1406/1406 [==============================] - 369s 263ms/step - loss: 1.1404 - accuracy: 0.5858 - val_loss: 1.1374 - val_accuracy: 0.5601 - _timestamp: 1659684946.4295 - _runtime: 927.7304
Epoch 3/100
1406/1406 [==============================] - 369s 262ms/step - loss: 1.1407 - accuracy: 0.5865 - val_loss: 1.1374 - val_accuracy: 0.5601 - _timestamp: 1659685315.5103 - _runtime: 1296.8112
Epoch 4/100
1406/1406 [==============================] - 368s 262ms/step - loss: 1.1415 - accuracy: 0.5829 - val_loss: 1.1374 - val_accuracy: 0.5601 - _timestamp: 1659685683.5234 - _runtime: 1664.8243
Epoch 5/100
1406/1406 [==============================] - 368s 262ms/step - loss: 1.1412 - accuracy: 0.5816 - val_loss: 1.1374 - val_accuracy: 0.5601 - _timestamp: 1659686051.6932 - _runtime: 2032.9941
Epoch 6/1

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
bert_model.trainable = True

# 매개변수
lr = tf.keras.optimizers.schedules.CosineDecay(learning_rate, decay_steps=1000)
if args.optimizer == "sgd":
    optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
elif args.optimizer == "adam":
    optim = tf.keras.optimizers.Adam(learning_rate=lr)
    
if loss == "cc":
    # loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
    loss_function = tf.keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing)
elif loss == "fl":
    loss_function = tfa.losses.SigmoidFocalCrossEntropy()

# 모델실행
model.compile(optimizer=optim, loss=loss_function, metrics=["accuracy"])
# 파일경로
checkpoint_filepath=f"load_model/{parser.description}"
# 조기중단/체크 포인트
checkpoint_callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                       tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor="val_loss", save_best_only=True, save_weights_only=True)]

history = model.fit(train_ds, validation_data=val_ds, epochs=2, callbacks=[checkpoint_callback, WandbCallback()])

Epoch 1/2


1406/1406 [==============================] - ETA: 0s - loss: 0.8615 - accuracy: 0.7027

INFO:tensorflow:Assets written to: /content/wandb/run-20220805_072013-2zn9kmhx/files/model-best/assets


INFO:tensorflow:Assets written to: /content/wandb/run-20220805_072013-2zn9kmhx/files/model-best/assets
wandb: Adding directory to artifact (/content/wandb/run-20220805_072013-2zn9kmhx/files/model-best)... Done. 15.7s


1406/1406 [==============================] - 1074s 743ms/step - loss: 0.8615 - accuracy: 0.7027 - val_loss: 0.8674 - val_accuracy: 0.6971 - _timestamp: 1659688562.4575 - _runtime: 4543.7584
Epoch 2/2
1406/1406 [==============================] - ETA: 0s - loss: 0.8313 - accuracy: 0.7186

INFO:tensorflow:Assets written to: /content/wandb/run-20220805_072013-2zn9kmhx/files/model-best/assets


INFO:tensorflow:Assets written to: /content/wandb/run-20220805_072013-2zn9kmhx/files/model-best/assets
wandb: Adding directory to artifact (/content/wandb/run-20220805_072013-2zn9kmhx/files/model-best)... Done. 15.9s


1406/1406 [==============================] - 1033s 735ms/step - loss: 0.8313 - accuracy: 0.7186 - val_loss: 0.8674 - val_accuracy: 0.6971 - _timestamp: 1659689601.2254 - _runtime: 5582.5263


In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
test_ds = BertDataGenerator(X_test.values, None, batch_size=BATCH_SIZE, shuffle=False, include_targets=False)

In [ ]:
# 예측
pred_prob = []
for i in tqdm(range(len(test_ds)+1)):
    pred_prob.append(model.predict(test_ds[i]))
pred_prob = np.vstack(pred_prob)
pred = np.argmax(pred_prob, axis=1)

label_decoder = {val : key for key, val in label_unique.items()}
result = [label_decoder[result] for result in pred]

pd.Series(result).value_counts()

100%|██████████| 1563/1563 [07:14<00:00,  3.59it/s]


5    11984
2     9397
1     3056
4      563
dtype: int64

In [ ]:
submission = pd.read_csv("./drive/MyDrive/input/쇼핑몰 리뷰 평점 분류/sample_submission.csv")
submission["target"] = result
submission.to_csv("submission.csv", index=False)